In [4]:
import pyspark
from pyspark.sql import SparkSession
import os

## DEFINE SENSITIVE VARIABLES
NESSIE_URI = "http://nessie:19120/api/v1"
MINIO_HOST = 'http://minio:9000'
MINIO_ACCESS_KEY = "jeAncqX06SoMwYqb3boy"
MINIO_SECRET_KEY = "jR2iwRbXnSpQOj5Yg9Zxw1kNGOtJ1UvUKbXcqLGT"

jar_packages = [
  "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1",
  "org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.95.0",
  "software.amazon.awssdk:bundle:2.17.178",
  "software.amazon.awssdk:url-connection-client:2.17.178"
]

spark_extensions = [
  "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
  "org.projectnessie.spark.extensions.NessieSparkSessionExtensions"
]

# fs.s3a.path.style.access true
# dremio.s3.compat true
# fs.s3a.endpoint http://minio:9000

conf = (
    pyspark.SparkConf()
			.setAppName('iceberg_hello_world')
			.set('spark.jars.packages', ','.join(jar_packages))
			.set('spark.sql.extensions', ','.join(spark_extensions))
			.set('spark.sql.catalog.nessie', "org.apache.iceberg.spark.SparkCatalog")
			.set('spark.sql.catalog.nessie.uri', NESSIE_URI)
			.set('spark.sql.catalog.nessie.ref', 'main')
			.set('spark.sql.catalog.nessie.authentication.type', 'NONE')
			.set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
			.set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
			.set('spark.sql.catalog.nessie.s3.endpoint', MINIO_HOST)
			.set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
			.set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
			.set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
			.set('spark.hadoop.fs.s3a.path.style.access', 'true')
              .set('spark.hadoop.fs.s3a.connection.timeout', '600000')
			.set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
			.set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
			.set('spark.hadoop.fs.s3a.endpoint', MINIO_HOST)
		
)


    # .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    # .config("spark.hadoop.fs.s3a.access.key", "SEU_ACCESS_KEY") \
    # .config("spark.hadoop.fs.s3a.secret.key", "SEU_SECRET_KEY") \
    # .config("spark.hadoop.fs.s3a.endpoint", "http://SEU_MINIO_ENDPOINT:9000") \
    # .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    
## Start Spark Session
spark = SparkSession.builder.config(conf=conf).master("local").getOrCreate()
print("Spark Running")

## LOAD A CSV INTO AN SQL VIEW
csv_df = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2023.csv")
csv_df.createOrReplaceTempView("csv_open_2023")

## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
spark.sql("CREATE TABLE IF NOT EXISTS nessie.df_open_2023_lesson2 USING iceberg AS SELECT * FROM csv_open_2023").show()
spark.sql("SELECT COUNT(*) FROM nessie.df_open_2023_lesson2").show()

24/09/07 03:31:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
Spark Running
+---------+
|namespace|
+---------+
|  default|
|  test_db|
+---------+

+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|  test_db|   test_table|      false|
|         |csv_open_2023|       true|
+---------+-------------+-----------+



In [9]:
spark.sql("SELECT COUNT(*) FROM nessie.df_open_2023_lesson2").show()

+--------+
|count(1)|
+--------+
|  302231|
+--------+



In [3]:
## LOAD A CSV INTO AN SQL VIEW
csv_df = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2023.csv")
csv_df.createOrReplaceTempView("csv_open_2023")

## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
spark.sql("CREATE TABLE IF NOT EXISTS nessie.df_open_2023_lesson2 USING iceberg AS SELECT * FROM csv_open_2023").show()

## QUERY THE ICEBERG TABLE
spark.sql("SELECT * FROM nessie.df_open_2023_lesson2 limit 10").show()

## QUERY THE COUNT OF ENTRIES
spark.sql("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2").show()

## CREATE A BRANCH WITH NESSIE
spark.sql("CREATE BRANCH IF NOT EXISTS lesson2 IN nessie")

## SWTICH TO THE NEW BRANCH
spark.sql("USE REFERENCE lesson2 IN nessie")

## DELETE ALL RECORDS WHERE countryOfOriginCode = 'FR'
spark.sql("DELETE FROM nessie.df_open_2023_lesson2 WHERE countryOfOriginCode = 'FR'")

## QUERY THE COUNT OF ENTRIES
spark.sql("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2").show()

## SWITCH BACK TO MAIN BRANCH
spark.sql("USE REFERENCE main IN nessie")

## QUERY THE COUNT OF ENTRIES
spark.sql("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2").show()

## MERGE THE CHANGES
spark.sql("MERGE BRANCH lesson2 INTO main IN nessie")

## QUERY THE COUNT OF ENTRIES
spark.sql("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2").show()

AnalysisException: Path does not exist: file:/home/docker/datasets/df_open_2023.csv